In [1]:
from vllm import LLM, SamplingParams
import numpy as np 
import pandas as pd 
import json 
from tqdm.notebook import tqdm
import gradio as gr
from user_info import get_user_info
import ast, html, re

In [1]:
sampling_params = SamplingParams(temperature=0, max_tokens=4096)
llm = LLM(model="lakshay/work-model")

NameError: name 'SamplingParams' is not defined

In [3]:
eval_df = pd.read_csv('../custom_data/iimjobs_eval_df.csv')

In [4]:
eval_df

,id,resume,current_designation,user_experience,keywords
0,1447358,ROHAN\nAZAD\nI am a certified data scientist w...,Data Analyst,6,Finance Analytics Excel Social_Media Leadershi...
1,1936688,P U N E E T G U P T A\nE-mail: puneet_mkgupta@...,Technical Specialist,13,Production Client SAP Testing Automation Team_...
2,690952,"Nayana Rao (B.E, PGDSM-MIT, SPJIMR - Virginia ...",Business consultant,12,Analysis Insurance Client Business_Process Com...
3,1394972,vz\nNITIN MISHRA\nBUSINESS HEAD\nOVERVIEW\n Ec...,Business Head,5,Sales Corporate Retail Planning Ecommerce Corp...
4,950206,"Anshul Garg\nCA, CPA(USA)\n+91-7982351846\nCaa...",Manager Finance,7,TAX Financial Audit Accountant Internal_Contro...
...,...,...,...,...,...
444,941147,CAREER OBJECTIVE\nCompany Experience\nAayush R...,Research Analyst - IT Sector,6,Financial IT Security Analyst Client Research ...
445,2180638,Versatile business leader bringing 18 years of...,CEO,18,Sales Tamil Marketing Sales_And_Marketing Agri...
446,238690,MANGAL DUTT TRIPATHI\nSEASONED MID LEVEL HR PR...,Regional HR Business Partner,14,HR Acquisition Finance Talent_Acquisition Attr...
447,442828,Nishant Mehta\nObjective\nTo work with and sub...,Zonal Business Head,19,Sales Marketing Brand Planning Distribution Pr...


In [14]:
def convert_to_json(input_string):
    # Replace single quotes at the start and end of keys and values with double quotes
    # This regex specifically targets the start of keys/values and the end of keys/values
    corrected_string = re.sub(r"(\{|\,)\s*\'", r'\1 "', input_string)  # Start of key/value
    corrected_string = re.sub(r"\'\s*(\,|\})", r'" \1', corrected_string)  # End of key/value
    corrected_string = re.sub(r"\'\s*:", r'":', corrected_string)  # Key end
    corrected_string = re.sub(r":\s*\'", r': "', corrected_string)  # Value start

    try:
        # Convert the string to a valid JSON
        valid_json = json.loads(corrected_string)
        print(f'it worked!')
        return valid_json
    except json.JSONDecodeError as e:
        print(f'Error thrown in JSON converter is {e}')
        return input_string

def make_eval_prompt(raw_text):
    
    work_format = '''
        [{"company": "company Name 1",
        "role": "job designation 1",
        "start_date": "mm/yyyy",
        "end_date": "mm/yyyy",
        "description": "complete Job description taken from resume"},
        {"company": "company name 2",
        "role": "job designation 2",
        "start_date": "mm/yyyy",
        "end_date": "mm/yyyy",
        "description": "complete Job description taken from resume"}]'''
    
    eval_prompt = f'''
    You are a helpful language model working for a job platform. You will be given the raw 
     unstructured text of a user's resume, and the task is to extract the work experience of the 
     user from the raw text in the following format as a properly formatted JSON which can be
     properly parsed. ONLY use double quotes while generating the output JSON : \n{{work_format}}\n.
    
     This is the resume text:\n{{resume_text}}\n
     This is the output in the required format:\n
    '''
    
    
    eval_prompt = eval_prompt.format(work_format=work_format,
                                     resume_text=raw_text)

    return eval_prompt

def parse_user_work_ex(info_json):
    work_ex = []
    try:
        for x in info_json['professional_info'][0]:
            
            user_dict = {}
            exp_dict = x
        
            keys = ['id','designation','fromExpMonth','fromExpYear','toExpMonth','toExpYear'] 
        
            for k in keys:
                user_dict[k] = exp_dict[k]
                
            user_dict['company'] = exp_dict['organization']['name']
            work_ex.append(user_dict)
    except:
        return work_ex
            
    return work_ex



def get_response_from_model(user_id):


    es_output = get_user_info(user_id)
    try:
        resume_text = es_output['resume'][0]
    except Exception as e:
        pass
    work_ex = parse_user_work_ex(es_output)
    if resume_text:
        eval_prompt = make_eval_prompt(resume_text)
    
    outputs = llm.generate(eval_prompt, sampling_params)
    out_text = outputs[0].outputs[0].text
    ot = html.unescape(out_text)
    generated_text = ot
    
    try:
        out_json = ast.literal_eval(generated_text)
        return json.dumps(out_json,indent=4), json.dumps(work_ex,indent=4)

    except Exception as e:
        print(f'couldnt JSONify this {e}')
        gt = convert_to_json(generated_text)
        return json.dumps(gt, indent=4), json.dumps(work_ex,indent=4)

In [15]:
# a,b = get_response_from_model(145199)

In [16]:
# json.loads(a)


In [17]:
# json.loads(b)


In [18]:
eval_dict = {}

In [19]:
eval_dict

{}

In [20]:
# eval_df.sample(200).id.values

In [21]:


for uid in tqdm(eval_df.sample(200).id.values):
    a,b = get_response_from_model(uid)

    eval_dict[uid]=[a,b]
    
    # print(uid)

  0%|          | 0/200 [00:00<?, ?it/s]


Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.10s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:36<00:00, 36.09s/it]


couldnt JSONify this invalid decimal literal (<unknown>, line 1)
it worked!



Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.27s/it]


couldnt JSONify this unterminated string literal (detected at line 1) (<unknown>, line 1)
it worked!


...

couldnt JSONify this invalid syntax (<unknown>, line 0)
Error thrown in JSON converter is Expecting value: line 1 column 1 (char 0)



Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.15s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:47<00:00, 47.87s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.80s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.34s/it]


couldnt JSONify this unexpected indent (<unknown>, line 2)
Error thrown in JSON converter is Expecting value: line 2 column 2 (char 2)



Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.02s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:46<00:00, 46.64s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:39<00:00, 39.86s/it]


UnboundLocalError: local variable 'resume_text' referenced before assignment

In [22]:
import pickle
with open('../custom_data/eval_dict_large.pkl','wb') as f:
    pickle.dump(eval_dict,f)

In [3]:
# demo = gr.Interface(
#     fn=get_response_from_model,
#     inputs=[gr.Textbox(label="UserID")],
#     outputs=[gr.Textbox(label="LLM Output"),gr.Textbox(label="User Info from ElasticSearch")],
#     allow_flagging="manual"
# )

# demo.launch(share=True)